In [ ]:
import numpy as np
import pandas as pd
from ConfigSpace import ConfigurationSpace, Configuration
from smac import MultiFidelityFacade, Scenario
from functools import partial


from datasets import get_hits
from metrics.tracks import track_metrics
from segment.track import gen_seg_track_layered
from segment.candidate import gen_seg_layered
from hopfield.energy.cross import cross_energy_matrix
from hopfield.energy.curvature import find_consecutive_segments, curvature_energy_matrix, prep_curvature
from hopfield.iterate import annealing_curve, anneal, update_act_bulk, update_act_sequential
from optimize import CONFIG_DEFAULTS

In [ ]:
import vispy
# selecting sub-backend behind jupyter_rfb:
# 1) pyglfw has high priority if present
# 2) to use any other backend, install then designate as default
# 4) use jupyter_rfb, it then selects the sub-backend according to config
# 5) enable jupyterlab extensions for interactive widget provided by jupyter_rfb

# only needed in jhub2, jhub and jlab-hpc can't run opengl at all, locally auto-selected backend is OK
vispy.config.update(default_backend='egl')
vispy.use('jupyter_rfb')

In [ ]:
from metrics.segments import gen_perfect_act
import vispy.scene
from hopfield.plot import _result_view

# Learning rate vs sequential update?
- small easy dataset
- many trials are needed to find a workable config

In [ ]:
from datasets import bman
N_EVENTS = 50
hits = get_hits('bman', n_events=N_EVENTS)
hits[['x', 'y', 'z']] /= bman.LAYER_DIST
hits

In [ ]:
hits = hits[hits['track']!=-1]
hits

In [ ]:
hits = hits.groupby(['event_id', 'track']).apply(lambda g: g if len(g)>2 else None).reset_index(drop=True)
hits

In [ ]:
def preprocess(g):
    event = g.reset_index(drop=True)
    seg = gen_seg_layered(event)
    pairs, cosines, r1, r2 = prep_curvature(event[['x', 'y', 'z']].to_numpy(), seg)
    return {
        'event': event,
        'seg': seg,
        'pairs': pairs,
        'cosines': cosines,
        'r1': r1,
        'r2': r2,
        'cross_matrix': cross_energy_matrix(seg),
    }
geometry = hits.groupby('event_id').apply(preprocess)

extra_conf = {
    'cooling_steps': 50,
    'rest_steps': 5,
}

In [ ]:
def dict_to_mem_usage(d):
    rows = {}
    for k,v in d.items():
        if isinstance(v, pd.DataFrame):
            rows[k] = v.memory_usage(deep=True).sum()
        elif isinstance(v, np.ndarray):
            rows[k] = v.nbytes
        else:
            rows[k] = v.data.nbytes + v.indptr.nbytes + v.indices.nbytes
    return rows

pd.DataFrame([dict_to_mem_usage(g) for g in geometry]).sum()/1024/1024

## bulk update, no learning rate

In [ ]:
# need many trials to find any working config
N_TRIALS = 200

scenario = Scenario(
    ConfigurationSpace({
        'alpha': (0., 1000.),
        'gamma': (0., 2000.),
        'bias': (-200.0, 200.0),
        'cosine_power': (0.0, 50.0),
        'cosine_min_rewarded': (0., 1.),
        't_max': (1., 1000.),
        'initial_act': (0., 1.),
    }),
    'bulk-norate',
    n_trials=N_TRIALS,
    min_budget=1,
    max_budget=N_EVENTS
)


In [ ]:
def evaluate(config: Configuration, seed: int, budget: float) -> float:
    conf = CONFIG_DEFAULTS.copy()
    conf.update(extra_conf)
    conf.update(config)
    rng = np.random.default_rng(seed=seed)
    scores = []
    for eid in rng.choice(geometry.index, int(budget), replace=False):
        event, seg, pairs, cosines, r1, r2, crossing_matrix = geometry[eid].values()
        curvature_matrix = curvature_energy_matrix(
            len(seg), pairs, cosines, r1, r2,
            cosine_power=conf['cosine_power'], cosine_threshold=conf['cosine_min_rewarded'],
            distance_power=conf['distance_power']
        )
        energy_matrix = conf['alpha'] * crossing_matrix - conf['gamma'] * curvature_matrix
        temp_curve = annealing_curve(conf['t_min'], conf['t_max'],
                                     conf['cooling_steps'], conf['rest_steps'])
        act = np.full(len(seg), conf['initial_act'])
        update_act = partial(update_act_bulk, learning_rate=conf['learning_rate'])
        for _ in anneal(energy_matrix, temp_curve, act, update_act, bias=conf['bias']):
            pass
        tseg = gen_seg_track_layered(event)
        score = track_metrics(event, seg, tseg, act, act>conf['threshold'])
        score['total steps'] = conf['cooling_steps'] + conf['rest_steps']
        score['trackml loss'] = 1. - score['trackml score']
        scores.append(score)
    return pd.DataFrame(scores).mean()['trackml loss']


In [ ]:
%%prun -s cumulative
optimizer = MultiFidelityFacade(scenario, evaluate, overwrite=True)
best_config = optimizer.optimize()

In [ ]:
optimizer.validate(best_config)

In [ ]:
best_config

In [ ]:
pd.DataFrame(optimizer.intensifier.trajectory)

In [ ]:
conf = CONFIG_DEFAULTS.copy()
conf.update(extra_conf)
conf.update(best_config)
event, seg, pairs, cosines, r1, r2, crossing_matrix = geometry.iloc[6].values()
curvature_matrix = curvature_energy_matrix(
    len(seg), pairs, cosines, r1, r2,
    cosine_power=conf['cosine_power'], cosine_threshold=conf['cosine_min_rewarded'],
    distance_power=conf['distance_power']
)
energy_matrix = conf['alpha'] * crossing_matrix - conf['gamma'] * curvature_matrix
temp_curve = annealing_curve(conf['t_min'], conf['t_max'],
                             conf['cooling_steps'], conf['rest_steps'])
act = np.full(len(seg), conf['initial_act'])
update_act = partial(update_act_bulk, learning_rate=conf['learning_rate'])
for _ in anneal(energy_matrix, temp_curve, act, update_act, bias=conf['bias']):
    pass
tseg = gen_seg_track_layered(event)

perfect_act = gen_perfect_act(seg, tseg)

result_canvas = vispy.scene.SceneCanvas(bgcolor='white', size=(300, 750))
positive = act >= conf['threshold']
result_canvas.central_widget.add_widget(
    _result_view(event, seg, act, perfect_act, positive, 
                 camera=vispy.scene.cameras.TurntableCamera(fov=20, scale_factor=8))
)    
result_canvas

## bulk update, optimize learning rate

In [ ]:
scenario = Scenario(
    ConfigurationSpace({
        'alpha': (0., 1000.),
        'gamma': (0., 2000.),
        'bias': (-40.0, 40.0),
        'cosine_power': (0.0, 50.0),
        'cosine_min_rewarded': (0., 1.),
        't_max': (1., 1000.),
        'initial_act': (0., 1.),
        'learning_rate': (0., 1.)
    }),
    'bulk-rate',
    n_trials=N_TRIALS,
    min_budget=3,
    max_budget=N_EVENTS
)

In [ ]:
optimizer = MultiFidelityFacade(scenario, evaluate, overwrite=True)
best_config = optimizer.optimize()

In [ ]:
optimizer.validate(best_config)

In [ ]:
best_config

In [ ]:
pd.DataFrame(optimizer.intensifier.trajectory)

In [ ]:
conf = CONFIG_DEFAULTS.copy()
conf.update(extra_conf)
conf.update(best_config)
event, seg, pairs, cosines, r1, r2, crossing_matrix = geometry.iloc[1].values()
curvature_matrix = curvature_energy_matrix(
    len(seg), pairs, cosines, r1, r2,
    cosine_power=conf['cosine_power'], cosine_threshold=conf['cosine_min_rewarded'],
    distance_power=conf['distance_power']
)
energy_matrix = conf['alpha'] * crossing_matrix - conf['gamma'] * curvature_matrix
temp_curve = annealing_curve(conf['t_min'], conf['t_max'],
                             conf['cooling_steps'], conf['rest_steps'])
act = np.full(len(seg), conf['initial_act'])
update_act = partial(update_act_bulk, learning_rate=conf['learning_rate'])
for _ in anneal(energy_matrix, temp_curve, act, update_act, bias=conf['bias']):
    pass
tseg = gen_seg_track_layered(event)

perfect_act = gen_perfect_act(seg, tseg)

result_canvas = vispy.scene.SceneCanvas(bgcolor='white', size=(300, 750))
positive = act >= conf['threshold']
result_canvas.central_widget.add_widget(
    _result_view(event, seg, act, perfect_act, positive, 
                 camera=vispy.scene.cameras.TurntableCamera(fov=20, scale_factor=8))
)    
result_canvas

## sequential update

In [ ]:
scenario = Scenario(
    ConfigurationSpace({
        'alpha': (0., 1000.),
        'gamma': (0., 2000.),
        'bias': (-40.0, 40.0),
        'cosine_power': (0.0, 50.0),
        'cosine_min_rewarded': (0., 1.),
        't_max': (1., 1000.),
        'initial_act': (0., 1.)
    }),
    'sequential',
    n_trials=N_TRIALS,
    min_budget=3,
    max_budget=N_EVENTS
)

In [ ]:
def evaluate(config: Configuration, seed: int, budget: float) -> float:
    conf = CONFIG_DEFAULTS.copy()
    conf.update(extra_conf)
    conf.update(config)
    rng = np.random.default_rng(seed=seed)
    scores = []
    for eid in rng.choice(geometry.index, int(budget), replace=False):
        event, seg, pairs, cosines, r1, r2, crossing_matrix = geometry[eid].values()
        curvature_matrix = curvature_energy_matrix(
            len(seg), pairs, cosines, r1, r2,
            cosine_power=conf['cosine_power'], cosine_threshold=conf['cosine_min_rewarded'],
            distance_power=conf['distance_power']
        )
        energy_matrix = conf['alpha'] * crossing_matrix - conf['gamma'] * curvature_matrix
        temp_curve = annealing_curve(conf['t_min'], conf['t_max'],
                                     conf['cooling_steps'], conf['rest_steps'])
        act = np.full(len(seg), conf['initial_act'])
        update_act = update_act_sequential
        for _ in anneal(energy_matrix, temp_curve, act, update_act, bias=conf['bias']):
            pass
        tseg = gen_seg_track_layered(event)
        score = track_metrics(event, seg, tseg, act, act>conf['threshold'])
        score['total steps'] = conf['cooling_steps'] + conf['rest_steps']
        score['trackml loss'] = 1. - score['trackml score']
        scores.append(score)
    return pd.DataFrame(scores).mean()['trackml loss']

In [ ]:
optimizer = MultiFidelityFacade(scenario, evaluate, overwrite=True)
best_config = optimizer.optimize()

In [ ]:
optimizer.validate(best_config)

In [ ]:
best_config

In [ ]:
pd.DataFrame(optimizer.intensifier.trajectory)

In [ ]:
conf = CONFIG_DEFAULTS.copy()
conf.update(extra_conf)
conf.update(best_config)
event, seg, pairs, cosines, r1, r2, crossing_matrix = geometry.iloc[1].values()
curvature_matrix = curvature_energy_matrix(
    len(seg), pairs, cosines, r1, r2,
    cosine_power=conf['cosine_power'], cosine_threshold=conf['cosine_min_rewarded'],
    distance_power=conf['distance_power']
)
energy_matrix = conf['alpha'] * crossing_matrix - conf['gamma'] * curvature_matrix
temp_curve = annealing_curve(conf['t_min'], conf['t_max'],
                             conf['cooling_steps'], conf['rest_steps'])
act = np.full(len(seg), conf['initial_act'])
update_act = update_act_sequential
for _ in anneal(energy_matrix, temp_curve, act, update_act, bias=conf['bias']):
    pass
tseg = gen_seg_track_layered(event)

perfect_act = gen_perfect_act(seg, tseg)

result_canvas = vispy.scene.SceneCanvas(bgcolor='white', size=(300, 750))
positive = act >= conf['threshold']
result_canvas.central_widget.add_widget(
    _result_view(event, seg, act, perfect_act, positive, 
                 camera=vispy.scene.cameras.TurntableCamera(fov=20, scale_factor=8))
)    
result_canvas